<h1> <p style="text-align: center;"> Исследование надежности заемщиков банка </p></h1>
<h3> Содержание </h3>

<ol class="toc-item">
        <li> Обзор данных </li>
        <li> Обработка данных </li>
        <ul>
            <li> заполнение пропусков </li>
            <li> проверка данных на аномалии </li>
            <li> изменение типов данных </li>
            <li> удаление дубликатов </li>
            <li> создание дополнительных словарей, декомпозиция исходных данных </li>
            <li> категоризация данных </li>
        </ul> 
        <li> Поиск зависимостей между параметрами </li>
        <ul>
            <li> взаимосвязь между количеством детей и благонадёжностью заёмщиков </li>
            <li> влияние семейного положения на возрат кредита в срок </li>
            <li> взаимосвязь между уровнем дохода и благонадёжностью заёмщиков </li>
            <li> влияние различных целей кредита на его возврат в срок </li>
        </ul>     
        <li> Общие выводы </li>
    </ol>

### Обзор данных

In [1]:
#открываем таблицу и изучаем общую информацию о данных, выводим первые 10 строк таблицы
import pandas as pd

debtors_database = pd.read_csv('') # в '' указываем наименование/ссылку на файл с данными
display(debtors_database.info())
debtors_database.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
debtors_database.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


С помощью функции describe() обнаружили некоторые особенности в наборе данных:
- в параметрах days_employed, total_income довольно много пропущенных значений (примерно 10% от общего количества), об этом "говорит" столбец count;
- в параметрах days_employed, dob_years, total_income очень большие отклонения от средних значениях, об этом "говорит" столбец std, этот факт свидетельствует о том, что в данных параметрах очень большой разброс данных, соответственно, средние значения данных параметров не стоит учитывать при анализе данных
- в параметрах children, days_employed очевидно есть некорретные значения, так как минимальные значения параметров - отрицательные (-1 ребёнок и -18 тыс. дней стажа), чего быть не может, а максимальные значения - очень высокие (20 детей и более 400 тыс. дней стажа(~ 1000 лет))

### Обработка данных
#### - заполнение пропусков

In [3]:
#в прошлом шаге с помощью медода info() нашли два столбца с пропущенными знчениями, проверяем, какую 
# долю составляют пропущенные значения в каждом из столбцов с пропусками;
days_employed_nan_part = len(debtors_database[debtors_database['days_employed'].isna()]) / len(debtors_database['days_employed'])
display(f'Доля пропущенных значений в столбце "days_employed" составляет {days_employed_nan_part:.2%}')
                             
total_income_nan_part = len(debtors_database[debtors_database['total_income'].isna()]) / len(debtors_database['total_income'])                  
display(f'Доля пропущенных значений в столбце "total_income" составляет {total_income_nan_part:.2%}')

'Доля пропущенных значений в столбце "days_employed" составляет 10.10%'

'Доля пропущенных значений в столбце "total_income" составляет 10.10%'

In [4]:
#проверяем есть ли взаимосвязь между пропусками в в данных столбцах
display(debtors_database[(debtors_database['days_employed'].isna()) & (debtors_database['total_income'].isna())])
display(debtors_database[debtors_database['days_employed'].isna()])
debtors_database[debtors_database['total_income'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [5]:
#заполняем пропуски медианными значениями в столбце total income
debtors_database['total_income'] = debtors_database['total_income'].fillna(debtors_database['total_income'].median())

В двух столбцах ('days_employed' и 'total_income') были обнаружены пропущенные значения.
Пропущенные значения в каждом из столбцов с пропусками составляют почти 10% всех данных.
Причиной появления пропусков в данных может быть отсутствие информации о работе данных заёмщиков в принципе, так как у одних и тех же заёмщиков отсутствует информация как о количестве отработанных дней, так и об общем доходе.

Попробуем восстановить пропуски ориентируясь на характеристик заемщиков - возвраст, образование и т.п. Изучив данные в столбцах с помощью describe() и value_counts(), обнаружили, что у 10 человек возраст равен нулю, поскольку для такого количества данных 10 человек - относительно немного, избавимся от этих строк.

Также придадим более адекватный вид данным по количеству отработанных дней, так как у большинства заёмщиков в выборке стаж составлял порядка 400 тыс. дней, т.е.более 1000 лет, скорее всего, данные были не по количеству дней, а по количеству отработанных часов, поэтому разделим данные в столбце 'days_employed' на 24 часа. 

Далее разделим всех заёмщиков по возрастным группам и типу занятости, сразу избавимся от минусовых значений, чтобы в дальнейшем медианные значения для заполнения пропусков были более корректными. Предположим, что минус в данном случае возник случайно, умножим отрицательные данные в столбце 'days_employed' на -1.
    
Для того, чтобы оценить взаимосвязь между стажем работы и такими характеристиками, как возраст, образование и тип занятости, оформим сводную таблицу медианных значений стажа с помощью методов groupby() и agg().
    
Далее напишем функцию для заполнения столбца 'days_employed' медианными значениями, ориентируясь на возраст образование и тип занятости заёмщиков.

In [6]:
debtors_database = debtors_database[debtors_database['dob_years']!=0]
debtors_database['days_employed'] = debtors_database['days_employed']/24
debtors_database['income_type'].value_counts()

сотрудник          11064
компаньон           5065
пенсионер           3836
госслужащий         1453
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [7]:
def age_group(age):
    if age <= 33:
        return 'молодые люди'
    elif age <= 60:
        return 'люди среднего возраста'
    else:
        return 'люди пожилого возраста' 
debtors_database['age_group'] = debtors_database['dob_years'].apply(age_group)

def income_group(income_type):
    if income_type == 'сотрудник' or income_type == 'госслужащий':
        return 'наёмные работники'
    if income_type == 'компаньон' or income_type == 'предприниматель':
        return 'самозанятые'
    else:
        return 'на иждивении у государства'
debtors_database['income_group'] = debtors_database['income_type'].apply(income_group)

In [8]:
def avoiding_negative_numbers (row):
    days_employed = row['days_employed']    
    if days_employed < 0:
        return row['days_employed']*(-1)
    else:
        return row['days_employed']   

debtors_database['days_employed'] = debtors_database.apply(avoiding_negative_numbers, axis=1)

In [9]:
(
    debtors_database.groupby(['age_group','income_group','education_id'])
    .agg({'days_employed':'median'}).sort_values(by='days_employed')
)

days_employed
age_group              income_group               education_id               
молодые люди           самозанятые                2                 35.202564
люди пожилого возраста самозанятые                2                 38.639000
молодые люди           наёмные работники          2                 40.319845
                       самозанятые                3                 42.925819
                                                  0                 43.492410
                                                  1                 46.220826
                       наёмные работники          3                 46.503905
люди среднего возраста наёмные работники          3                 46.848294
молодые люди           наёмные работники          1                 49.133443
                                                  0                 50.308020
люди среднего возраста самозанятые                3                 52.068321
                                                  2                 64.752646
                       наёмные работники          2                 66.350368
                       самозанятые                0                 74.340443
                                                  1                 79.762798
                       наёмные работники          1                 84.281271
                                                  0                 93.032761
люди пожилого возраста самозанятые                1                 98.270213
                       наёмные работники          1                100.538193
                       самозанятые                0                107.338342
                       наёмные работники          0                135.393031
                                                  2                147.992841
люди среднего возраста наёмные работники          4                160.488961
люди пожилого возраста наёмные работники          3                168.889982
молодые люди           на иждивении у государства 0               8124.498315
                                                  1              14330.525564
люди пожилого возраста на иждивении у государства 4              14872.104898
люди среднего возраста на иждивении у государства 3              14883.453818
люди пожилого возраста на иждивении у государства 3              15095.102200
люди среднего возраста на иждивении у государства 1              15204.190664
                                                  0              15209.049986
люди пожилого возраста на иждивении у государства 1              15220.275944
                                                  2              15276.174641
                                                  0              15327.408606
люди среднего возраста на иждивении у государства 2              15510.437569

In [10]:
median_dependents_experience = (
    debtors_database[debtors_database['income_group']=='на иждивении у государства']['days_employed']
    .median()
)
median_employees_eldest_experience = (
    debtors_database[(
        debtors_database['income_group']=='наёмные работники') & (
        debtors_database['age_group']!='молодые люди')]['days_employed']
    .median()
)
median_lower_educated_experience = (
    debtors_database[(debtors_database['income_group']!='на иждивении у государства') & (
        debtors_database['education_id']!=4)]['days_employed']
    .median()
)

In [11]:
def median_days_employed(row):
    income_group = row['income_group']
    age_group = row['age_group']
    education_id = row['education_id']
    days_employed = row['days_employed']
    if income_group=='на иждивении у государства':
        return median_dependents_experience
    elif income_group=='наёмные работники' and age_group!='молодые люди': 
        return median_employees_eldest_experience
    elif income_group!='на иждивении у государства' and education_id!=4:
        return median_lower_educated_experience
    else:
        return row['days_employed']

In [12]:
debtors_database['days_employed'] = debtors_database.apply(median_days_employed, axis=1)


In [13]:
debtors_database.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,income_group
0,1,86.272344,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,люди среднего возраста,наёмные работники
1,1,86.272344,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,люди среднего возраста,наёмные работники
2,0,67.933106,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,молодые люди,наёмные работники
3,3,67.933106,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,молодые люди,наёмные работники
4,0,15213.167609,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,люди среднего возраста,на иждивении у государства
5,0,67.933106,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,молодые люди,самозанятые
6,0,67.933106,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,люди среднего возраста,самозанятые
7,0,86.272344,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,люди среднего возраста,наёмные работники
8,2,86.272344,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,люди среднего возраста,наёмные работники
9,0,86.272344,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,люди среднего возраста,наёмные работники


#### - проверка данных на аномалии

In [14]:
#выводим строки с аномальными данными (отрицательным количеством дней трудового стажа в столбце 'days_employed')
debtors_database[debtors_database['days_employed'] < 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,income_group


In [15]:
#создаём функцию для обработки значений в столбцах с аномалиями, обработка осуществляется путём замены отрицательных 
#значений на медианные значения 
def avoiding_anomaly(row):
    days_employed = row['days_employed']    
    if days_employed < 0:
        return debtors_database[debtors_database['days_employed'] > 0]['days_employed'].median()
    return row['days_employed']
    
#применяем данную функцию к нашему датафрейму, создаём новый столбец с корректными значениями о количестве дней 
#трудового стажа
debtors_database['correct_days_employed'] = debtors_database.apply(avoiding_anomaly, axis=1)

#проверяем обновлённый датафрейм
debtors_database.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,income_group,correct_days_employed
0,1,86.272344,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,люди среднего возраста,наёмные работники,86.272344
1,1,86.272344,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,люди среднего возраста,наёмные работники,86.272344
2,0,67.933106,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,молодые люди,наёмные работники,67.933106
3,3,67.933106,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,молодые люди,наёмные работники,67.933106
4,0,15213.167609,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,люди среднего возраста,на иждивении у государства,15213.167609
5,0,67.933106,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,молодые люди,самозанятые,67.933106
6,0,67.933106,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,люди среднего возраста,самозанятые,67.933106
7,0,86.272344,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,люди среднего возраста,наёмные работники,86.272344
8,2,86.272344,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,люди среднего возраста,наёмные работники,86.272344
9,0,86.272344,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,люди среднего возраста,наёмные работники,86.272344


В данных нам встретились артефакты (аномалии) — отрицательное количество дней трудового стажа в столбце 'days_employed'. 
Причиной, такой ошибки в данных, скорее всего, послужил тот факт, что общую продолжительность трудового стажа, как правило, подсчитывают довольно нестандартным способом: из трудовой книжки сначала выписывают все даты записей приема на работу и все даты увольнения, после чего их складывают, чтобы в результате получить две суммы: одну - из дат приема, вторую - из дат увольнений. Затем необходимо из второй суммы вычесть первую. Иногда, в таких случаях, разница между суммой дат увольнений и суммой дат приемов может получиться отрицательной (такой пример можно найти по этой ссылке https://www.kdelo.ru/art/384568-staj-raboty-18-m8), возможно,  и в данном случае, ошибка обусловлена особенностями подсчёта и несовсем корректной доработкой данных.
Поскольку установить изначальные даты увольнений/ приёмов мы не можем, то лучше заменить некорректные отрицательные данные на новое медианное значение на основе данных по другим заёмщикам, у которых положительное количество дней трудового стажа.

#### - изменение типов данных

In [16]:
#заменяем вещественный тип данных в столбце 'total_income' на целочисленный с помощью метода astype()
debtors_database['total_income'] = debtors_database['total_income'].astype('int')
debtors_database['correct_days_employed'] = debtors_database['correct_days_employed'].astype('float')
debtors_database.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   children               21424 non-null  int64  
 1   days_employed          21424 non-null  float64
 2   dob_years              21424 non-null  int64  
 3   education              21424 non-null  object 
 4   education_id           21424 non-null  int64  
 5   family_status          21424 non-null  object 
 6   family_status_id       21424 non-null  int64  
 7   gender                 21424 non-null  object 
 8   income_type            21424 non-null  object 
 9   debt                   21424 non-null  int64  
 10  total_income           21424 non-null  int64  
 11  purpose                21424 non-null  object 
 12  age_group              21424 non-null  object 
 13  income_group           21424 non-null  object 
 14  correct_days_employed  21424 non-null  float64
dtypes:

#### - у.даление дубликатов.

In [17]:
#обработка явных дубликатов, сначала находим количество строк-дубликатов, затем удаляем их 
display(debtors_database.duplicated().sum())
debtors_database = debtors_database.drop_duplicates()

54

In [18]:
#ищем неявные дубликаты
debtors_database['education'].value_counts()
#приводим данные в столбце к одному регистру
debtors_database['education'] = debtors_database['education'].str.lower()
#проверяем
debtors_database['education'].value_counts()

среднее                15124
высшее                  5216
неоконченное высшее      742
начальное                282
ученая степень             6
Name: education, dtype: int64

In [19]:
# снова проверяем данные на дубликаты
display(debtors_database.duplicated().sum())

17

In [20]:
debtors_database = debtors_database.drop_duplicates()
debtors_database.duplicated().sum()

0

В данных присутствовали строки-дубликаты в количестве 71 шт., были удалены.
Также обработали неявные дубликаты: привели данные в столбце 'education' к единому регистру. 
Также проверили остальные столбцы с данными типа 'object' на наличие неявных дубликатов, в столбце 'purpose' также дублируется однотипная информация, но с ним будем работать отдельно в дальнейшем.
В качестве метода для поиска и удаления явных дубликатов в данных были выбраны duplicated().sum() и drop_duplicates, эти методы позволяют "пройтись" сразу по всему датафрейму и удалить абсолютно идентичные данные.
Для поиска и обработки неявных дубликатов - value_counts() и str.lower(), данные методы удобны для обработки информации по отдельным столбцам, их удобно применять к данным типа 'object', где прочие варианты проверки могли бы дать некорректный результат.
Возможные причины появления дубликатов:
- для явных строк-дубликатов - неоднократное обращение одного и того человека за кредитом
- для неяных дубликатов - отсутствие единообразия при заполнении заявки на получение кредита, заполнение заявок разными операторами банка

#### - создание дополнительных словарей, декомпозиция исходных данных

In [21]:
#создаём "словари" с данными об образовании и семейном положении, сразу удаляем дубликаты для более удобного пользования
education_dict = debtors_database[['education_id', 'education']].drop_duplicates().reset_index(drop=True)
display(education_dict)
family_dict = debtors_database[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)
family_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [22]:
# удаляем из исходного датафрейма столбцы 'education' и 'family_status', оставляем только их 
# идентификаторы: 'education_id' и 'family_status_id', также удаляем столбец 'days_employed', 
# оставляем только 'correct_days_employed' с уже обаботанными данными по стажу заёмщиков
debtors_database = debtors_database[['children', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose', 'correct_days_employed']]
debtors_database.head()

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,correct_days_employed
0,1,42,0,0,F,сотрудник,0,253875,покупка жилья,86.272344
1,1,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,86.272344
2,0,33,1,0,M,сотрудник,0,145885,покупка жилья,67.933106
3,3,32,1,0,M,сотрудник,0,267628,дополнительное образование,67.933106
4,0,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,15213.167609


#### - категоризация данных

In [23]:
#создаём столбец 'total_income_category' с различными категориями заёмщиков по уровню доходов
def group_total_income(row):
    total_income = row['total_income']
    if total_income <= 30000:
        return 'E'
    elif 30001 <= total_income <= 50000:
        return 'D'
    elif 50001 <= total_income <= 200000:
        return 'C'
    elif 200001 <= total_income <= 1000000:
        return 'B'
    else:
        return 'A'
debtors_database['group_total_income'] = debtors_database.apply(group_total_income, axis=1)
debtors_database.head()

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,correct_days_employed,group_total_income
0,1,42,0,0,F,сотрудник,0,253875,покупка жилья,86.272344,B
1,1,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,86.272344,C
2,0,33,1,0,M,сотрудник,0,145885,покупка жилья,67.933106,C
3,3,32,1,0,M,сотрудник,0,267628,дополнительное образование,67.933106,B
4,0,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,15213.167609,C


In [24]:
#изучаем данные в столбце 'purpose', определяем ключевые слова для группировки данных в более крупные категории
debtors_database['purpose'].value_counts()

свадьба                                   786
на проведение свадьбы                     764
сыграть свадьбу                           760
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с коммерческой недвижимостью     648
операции с жильем                         646
покупка жилья                             640
жилье                                     640
покупка жилья для семьи                   637
строительство собственной недвижимости    633
недвижимость                              629
операции со своей недвижимостью           627
строительство жилой недвижимости          621
строительство недвижимости                619
покупка своего жилья                      619
покупка недвижимости                      618
ремонт жилью                              605
покупка жилой недвижимости                603
на покупку своего автомобиля              502
заняться высшим образованием      

In [25]:
#создаём функцию, которая на основании данных из столбца 'purpose' сформирует новый столбец 'purpose_category', 
#в который войдут следующие категории: 'операции с автомобилем', 'операции с недвижимостью', 'проведение свадьбы', 
#'получение образования'
def group_purpose(row):
    purpose = row['purpose']
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    elif 'жиль' in purpose or 'недвижимост' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образовани' in purpose:
        return 'получение образования'
debtors_database['group_purpose'] = debtors_database.apply(group_purpose, axis=1)
debtors_database.head()

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,correct_days_employed,group_total_income,group_purpose
0,1,42,0,0,F,сотрудник,0,253875,покупка жилья,86.272344,B,операции с недвижимостью
1,1,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,86.272344,C,операции с автомобилем
2,0,33,1,0,M,сотрудник,0,145885,покупка жилья,67.933106,C,операции с недвижимостью
3,3,32,1,0,M,сотрудник,0,267628,дополнительное образование,67.933106,B,получение образования
4,0,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,15213.167609,C,проведение свадьбы


### Поиск зависимостей между параметрами

#### - взаимосвязь между количеством детей и благонадёжностью заёмщиков

СОтветимгруппируем данные по количеству детей у заёмщиков, которые ввозвращают вредиты в срок(индекс 0) и у которых есть задолженности по кредитам(индекс 1), подсчитаем количество заёмщиков из обоих категорий по количеству детей у них.

In [26]:
debt_by_children = debtors_database.pivot_table(index=['children'], values='debt',aggfunc=['sum','count'])
debt_by_children['proportion_in_%'] = debt_by_children[(  'sum', 'debt')]/debt_by_children[('count', 'debt')]*100
debt_by_children

,sum,count,proportion_in_%
,debt,debt,
children,,,
-1,1,47,2.127660
0,1058,14022,7.545286
1,441,4792,9.202838
2,194,2039,9.514468
3,27,328,8.231707
4,4,41,9.756098
5,0,9,0.000000
20,8,75,10.666667


Вывели проценты по каждой категории: среди бездетных людей, процент должников меньше (7,5 %), процент должников у людей с детьми, выше - в среднем, от 9%, никакой тенденции в зависимости от увеличения/ уменьшения количества детей и благонадёжности заёмщиков не выявлено, только у людей с 3-мя детьми - благонадёжность немного выше, процент должников составляет 8,23%.

Обнаруживаем "интересный" факт, у 47 заёмщиков по минус одному ребёнку. Также крайне удивительным выглядит факт присутствия в данных 76 заёмщиков, у каждого из которых 20 детей.Очевидно, в эти данные закралась ошибка, но, поскольку уточнить, откуда она взялась, не представляется возможным, будем анализировать данные дальше без учёта этих 47 заёмщиков, в конце концов это всего 0,6% из общего числа заёмщиков. 
Из оставшихся данных получившейся таблицы можно сделать вывод, что количество детей у заёмщиков не имеет ключевого значения при оценке их благонадёжности. Так, на одного бездетного заёмщика с задолженностью по кредиту, приходится 12 бездетных заёмщиков, которые возвращают кредиты в срок. Те заёмщики, у которых дети есть, оказываются менее благонадёжными: на одного заёмщика с задодженностью приходится ориентировочно 9-10 порядочных заёмщиков, и доля заёмщиков с задолженностями, у которых дети есть, не имеет тенденции ни к росту, ни к снижению пропорционально количеству детей, наблюдается "прорыв" только у заёмщиков, имеющих по трое детей, там на 1 неблагонадёжного заёмщика приходится 11 благонадёжных. Но в общем и целом, выходит, что заёмщики с детьми менее благонадёжны, чем заёмщики без детей.

#### - влияние семейного положения на возрат кредита в срок

Аналогично сгруппируем оба вида заёмщиков по их семейному положению.

In [27]:
debt_by_family_status = debtors_database[['family_status_id', 'debt']].merge(family_dict, on='family_status_id', how='right')
debt_by_family_status = debt_by_family_status.pivot_table(index=['family_status'], values='debt',aggfunc=['sum','count'])
debt_by_family_status['proportion_in_%'] = debt_by_family_status[(  'sum', 'debt')]/debt_by_family_status[('count', 'debt')]*100
debt_by_family_status

,sum,count,proportion_in_%
,debt,debt,
family_status,,,
Не женат / не замужем,273,2794,9.770938
в разводе,85,1185,7.172996
вдовец / вдова,62,954,6.498952
гражданский брак,386,4130,9.346247
женат / замужем,927,12290,7.542718


Анализ данной таблицы показывает, что несемейные люди и люди, живущие в гражданском браке, менее благонадёжные заёмщики (на одного заёмщика, имеющего задолженность, приходится 9 заёмщиков, которые возвращают кредиты в срок). Наиболее благонадёжные заёмщики оказались среди вдовцов и вдов (14 благонадёжных на 1 неблагонадёжного), далее идут люди в разводе(13/1) и люди, состоящие в браке(12/1).

#### - взаимосвязь между уровнем дохода и благонадёжностью заёмщиков

Группируем оба вида заёмщиков по их доходу.

In [28]:
debt_by_income = debtors_database.pivot_table(index=['group_total_income'], values='debt',aggfunc=['sum','count'])
debt_by_income['proportion_in_%'] = debt_by_income[(  'sum', 'debt')]/debt_by_income[('count', 'debt')]*100
debt_by_income

,sum,count,proportion_in_%
,debt,debt,
group_total_income,,,
A,2,25,8.000000
B,355,5015,7.078764
C,1353,15943,8.486483
D,21,348,6.034483
E,2,22,9.090909


В группе А у нас люди с наибольшим уровнем дохода, в группе Е - с наименьшим. Показательно, что среди людей с наименьшим уровнем дохода меньше всего благонадёжных заёмщиков(10 к одному), немного больше их в группе С со средним уровнем дохода (10,8 к одному) и ещё немного больше в группе с наибольшим уровнем дохода(11,5 к одному). Но самыми благонадёжными заёмщиками оказались люди с доходом от 30 до 50 тысяч (группа D), тут на одного неблагонадёжного заёмщика приходится 15,7 надёжных. Следующая по благонадёжности - группа В (от 200 тыс. до 1 млн), тут на одного неблагонадёжного заёмщика приходится 13,2 надёжных.

#### - влияние различных целей кредита на его возврат в срок

Группируем заёмщиков по различным типам целей.

In [29]:
debt_by_purpose = debtors_database.pivot_table(index=['group_purpose'], values='debt',aggfunc=['sum','count'])
debt_by_purpose['proportion_in_%'] = debt_by_purpose[(  'sum', 'debt')]/debt_by_purpose[('count', 'debt')]*100
debt_by_purpose

,sum,count,proportion_in_%
,debt,debt,
group_purpose,,,
операции с автомобилем,400,4284,9.337068
операции с недвижимостью,779,10764,7.237087
получение образования,370,3995,9.261577
проведение свадьбы,184,2310,7.965368


Больше всего кредитов заёмщики берут для осуществления операций с недвижимостью, меньше всего - на проведение свадеб.
При этом на одного человека, который имеет задолженность по свадебному кредиту, приходится 11,6 человек, которые рассчитываются за такой кредит вовремя. По кредитам на недвижимость ситуация выглядит лучше всего (12,8 к одному).Хуже всего обстоят дела с кредитами на образование (9,8 к одному) и на автомобили(9,7 к одному).

Подытожим полученные данные:

1) Несемейные люди и люди, живущие в гражданском браке, менее благонадёжные заёмщики (9,8% и 9,3% должников). Наиболее благонадёжные заёмщики оказались среди вдовцов и вдов (6,5% должников), далее идут люди в разводе(7,2%) и люди, состоящие в браке(7,5%).

2) Среди людей с наименьшим уровнем дохода меньше всего благонадёжных заёмщиков (9,1% должников), немного больше их в группе С со средним уровнем дохода (8,5% должников) и ещё немного больше в группе с наибольшим уровнем дохода(8% должников). Самые благонадёжные заёмщики среди людей с доходом от 30 до 50 тысяч (группа D), тут примерно 6% должников, следующая по благонадёжности - группа В (от 200 тыс. до 1 млн), тут 7,1% должников.

3) Среди людей, которые берут кредиты на образование и автомобили больше всего должников (примерно по 9,3% должников), по свадебным кредитам - примерно 8% должников, меньше всего должников по ипотечным кредитам - всего 7,2%.
</div>

## Общий вывод:

В данном проекте был проведён анализ зависимости возврата кредита в срок от н/у факторов:
- количество детей у заёмщика
- семейное положение заёмщика
- уровень его дохода
- цель кредитования

Анализ приводит к довольно неожиданным выводам:
- наиболее надёжными заёмщиками являются бездетные вдовцы и вдовы с уровнем дохода от 30 до 50 тысяч, которые берут кредиты для операций с недвижимостью
- наименее надёжные заёмщики - среди холостых людей с детьми и уровнем дохода до 30 тысяч, которые берут кредиты для операций с автомобилем.
Возможно, имеет смысл перепроверять и соотносить данные не только по отдельным критериям, а по их совокупности.
Ради интереса можно поискать в нашем датафрейме самых надёжных и самых ненадёжных заёмщиков по совокупности проанализированных параметров, есть вероятность, что таких людей, в целом, окажется немного.
Проверка показала, что самых надёждых заёмщиков в нашем датафрейме 11, а вот самых ненадёжных - нет совсем.

In [30]:
debtors_database[(debtors_database['children']==0) & (debtors_database['family_status_id']==2) & (debtors_database['group_total_income']=='D') & (debtors_database['group_purpose']=='операции с недвижимостью')].groupby('debt')['debt'].count()

debt
0    11
Name: debt, dtype: int64

In [31]:
debtors_database[(debtors_database['children']>0) & (debtors_database['family_status_id']==1) & (debtors_database['group_total_income']=='С') & (debtors_database['group_purpose']=='получение образования')].groupby('debt')['debt'].count()

Series([], Name: debt, dtype: int64)